In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv("../input/spaceship-titanic/train.csv")
test_df = pd.read_csv("../input/spaceship-titanic/test.csv")

train_ID = train_df['PassengerId'].astype(str)
test_ID = test_df['PassengerId'].astype(str)

In [ ]:
from sklearn import preprocessing

fitTransformer = preprocessing.LabelEncoder()
train_df = train_df.apply(fitTransformer.fit_transform)
test_df = test_df.apply(fitTransformer.fit_transform)

In [ ]:
X = train_df.copy()
X.drop(['PassengerId','Name'], axis = 1)
y = X.pop('Transported')

X_test = test_df.copy()
X_test.drop(['PassengerId','Name'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = \
    train_test_split(X, y, stratify=y, train_size=0.80)
input_shape = [X_train.shape[1]]

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

spaceShip_model = keras.Sequential([
                layers.BatchNormalization(input_shape=input_shape),
                layers.Dense(64, activation='relu'),
                layers.Dropout(0.3),
                layers.BatchNormalization(),
                layers.Dense(256, activation='relu'),
                layers.Dropout(0.2),
                layers.BatchNormalization(),
                layers.Dense(64, activation='relu'),
                layers.Dropout(0.1),
                layers.Dense(1, activation='sigmoid'),
                
])

In [ ]:
spaceShip_model.compile(
                optimizer ='adam',
                loss= "binary_crossentropy",
                metrics=['binary_accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
                 patience= 30,
                 min_delta= 0.0001,
                 restore_best_weights= True)

In [ ]:
history = spaceShip_model.fit(
            X_train, y_train,
            validation_data = (X_valid, y_valid),
            batch_size = 512,
            epochs= 100,
            callbacks= [early_stopping]
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot(title="Cross-entropy")
history_df.loc[:, ['binary_accuracy', 'val_binary_accuracy']].plot(title="Accuracy")

In [ ]:
valid_scores = spaceShip_model.evaluate(X_valid, y_valid)

In [ ]:
y_test = spaceShip_model.predict(X_test)
y_test = (y_test>0.5).T
print(y_test.shape)

In [ ]:
sample_DF = pd.DataFrame([test_ID]).T;
sample_DF.columns = ["PassengerId"]
sample_DF= sample_DF.applymap(str)
sample_DF ['Transported'] = y_test.T
sample_DF.to_csv("submission.csv",index=False)
sample_DF.head()
